<a href="https://colab.research.google.com/github/AICFT-Miyata/AICFT/blob/main/semantic_search_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install fugashi ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 70.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.9/694.9 kB 45.1 MB/s eta 0:00:00
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=9c7e0de20d5dd9e0f9ffdc8c8982177d51449ca4d336aa1770753da36456fa1b
  Stored in directory: /root/.cache/pip/wheels/93/8b/55/dd5978a069678c372520847cf84ba2ec539cb41917c00a2206
Successfully built ipadic


In [4]:
!pip install unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658817 sha256=c1d97848452e03a123d5d8ad6eaf92ca33ef049e199c7e0ce54f8fc9ab852dfa
  Stored in directory: /root/.cache/pip/wheels/5e/1f/0f/4d43887e5476d956fae828ee9b6687becd5544d68b51ed633d
Successfully built unidic-lite


In [19]:
!pip install google-genai

In [21]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gspread # スプレッドシートへの書き込み用
import time

# --- 1. 定数・初期設定 ---
# ⚠️ ここをあなたの環境に合わせて修正してください
SPREADSHEET_NAME = 'スプレッドシート作成検証'
TARGET_SHEET = '検索入力シート' # VBAから入力が行われるシート名
TRAINING_DATA_PATH = '/content/drive/MyDrive/AICFT/DB.csv'

# モデルのロード (以前成功した軽量モデル)
EMBEDDING_MODEL = SentenceTransformer('cl-tohoku/bert-base-japanese-whole-word-masking')

# --- 2. データの準備とエンベディング生成 ---
def load_and_embed_data(file_path):
    try:
        # CP932でファイルを読み込み、必要なカラムのみを選択
        df_train = pd.read_csv(file_path, encoding='utf-8', sep=',',
          usecols=['階級', '目標（50～100文字）', '自己申告（50～100文字）', '上司コメント（50～100文字）'])
        df_train.columns = ['Rank', 'Goal', 'SelfReport', 'SupervisorComment']
        df_train['CombinedText'] = df_train['Goal'].astype(str) + " [SEP] " + df_train['SelfReport'].astype(str)

        # エンベディング生成
        embeddings = EMBEDDING_MODEL.encode(df_train['CombinedText'].tolist(), convert_to_tensor=True)
        return df_train, embeddings.cpu().numpy()
    except Exception as e:
        print(f"データ処理エラー: {e}")
        return None, None

df_train, train_embeddings = load_and_embed_data(TRAINING_DATA_PATH)

# --- 3. 検索ロジック ---
def find_best_match_comment(input_goal, input_self_report):
    if df_train is None:
        return "ERROR: Training data not loaded.", 0.0
    new_combined_text = str(input_goal) + " [SEP] " + str(input_self_report)
    new_embedding = EMBEDDING_MODEL.encode(new_combined_text, convert_to_tensor=True).cpu().numpy().reshape(1, -1)
    similarities = cosine_similarity(new_embedding, train_embeddings)[0]
    best_match_index = np.argmax(similarities)
    best_comment = df_train.iloc[best_match_index]['SupervisorComment']
    return best_comment, similarities[best_match_index]


# --- 4. メイン処理 (Apps Scriptから実行される想定) ---
def main_search_and_write():
    if df_train is None:
        print("検索スキップ: 訓練データなし")
        return "検索スキップ: 訓練データなし", 0.0

    print("--- gspread認証開始 ---")
    # Colabでの認証画面を表示するための処理
    from google.colab import auth
    auth.authenticate_user()

    # 認証情報をgspreadに渡す (Colab推奨形式への修正)
    import google.auth # 新しくインポートを追加

    # 認証情報を取得
    creds, _ = google.auth.default()

    # 認証情報を使ってgspreadを初期化
    gc = gspread.authorize(creds)
    spreadsheet = gc.open(SPREADSHEET_NAME)
    input_sheet = spreadsheet.worksheet(TARGET_SHEET)

    # スプレッドシートから入力データを読み込み (B1: 目標, E1: 自己申告)
    input_data = input_sheet.get('B1:E1', value_render_option='UNFORMATTED_VALUE')[0]
    input_goal = input_data[0]
    input_self_report = input_data[3]

    print(f"入力データ: 目標='{input_goal}', 自己申告='{input_self_report}'")

    # 検索実行
    comment, similarity_score = find_best_match_comment(input_goal, input_self_report)

# 結果をスプレッドシートの F1 セルに出力 (データをリストのリスト形式に変更)
    input_sheet.update('F1', [[comment]])

    # G1 セルに出力
    input_sheet.update('G1', [[f"類似度: {similarity_score:.4f}"]])
    print(f"検索結果をF1に出力完了。類似度: {similarity_score:.4f}")

    return comment, similarity_score

if __name__ == "__main__":
    # 単体テストとして実行
    main_search_and_write()

--- gspread認証開始 ---
入力データ: 目標='積極的にワークライフバランスに努める', 自己申告='テレワークを５回実施した。'


/tmp/ipython-input-3421670774.py:80: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  input_sheet.update('F1', [[comment]])
/tmp/ipython-input-3421670774.py:83: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  input_sheet.update('G1', [[f"類似度: {similarity_score:.4f}"]])


検索結果をF1に出力完了。類似度: 0.8999


In [20]:
# --- 5. AIによるコメント洗練ロジック (ラッキーテスト) ---
def refine_comment_with_ai(input_goal, input_self_report, best_comment):
    try:
        print("--- AIによるコメント洗練開始 ---")
        # ⚠️ ラッキーテスト：APIキー不要のアクセスを試みます
        # 成功しなかった場合は、次のエラー (NameError/APIError) で止まります
        from google import genai

        # 認証（authenticate_userで得られた認証情報を使えるか試みる）
        # ただし、通常はAPIキーが必要
        client = genai.Client()

        prompt = f"""
        あなたは、人事評価の上司コメントを作成するAIです。
        以下の[入力データ]に基づき、[推薦コメント]を参考にしながら、より個性的で具体的、かつ前向きな「上司コメント」を日本語で作成してください。
        出力は洗練されたコメント本文のみとしてください。

        [入力データ]
        目標: {input_goal}
        自己申告: {input_self_report}

        [推薦コメント]
        {best_comment}
        """

        response = client.models.generate_content(
            model='gemini-2.5-flash', # 高速な軽量モデルを使用
            contents=prompt,
            # temperature=0.7 # コメント生成には創造性を許容
        )

        refined_comment = response.text.strip()
        print("AI洗練コメント生成成功。")
        return refined_comment

    except Exception as e:
        print(f"AI洗練エラー。通常通り検索結果を出力します。エラー: {e}")
        return best_comment # エラー時は検索結果をそのまま返す

# --- 4. メイン処理 (既存の main_search_and_write を修正) ---
# 既存の main_search_and_write 関数全体をこの新しいバージョンに置き換えてください
def main_search_and_write():
    if df_train is None:
        print("検索スキップ: 訓練データなし")
        return "検索スキップ: 訓練データなし", 0.0

    print("--- gspread認証開始 ---")
    from google.colab import auth
    auth.authenticate_user()

    import google.auth
    creds, _ = google.auth.default()
    gc = gspread.authorize(creds)
    spreadsheet = gc.open(SPREADSHEET_NAME)
    input_sheet = spreadsheet.worksheet(TARGET_SHEET)

    data_list = input_sheet.get('B1:E1', value_render_option='UNFORMATTED_VALUE')
    if not data_list or len(data_list[0]) < 4:
        print("ERROR: スプレッドシートのB1～E1範囲にデータが不足しています。処理を中断します。")
        return "ERROR: データ不足", 0.0

    input_data = data_list[0]
    input_goal = input_data[0]
    input_self_report = input_data[3]

    print(f"入力データ: 目標='{input_goal}', 自己申告='{input_self_report}'")

    # 1. 検索実行
    best_comment, similarity_score = find_best_match_comment(input_goal, input_self_report)
    print(f"検索結果 (原案): {best_comment}")

    # 2. AI洗練 (追加ステップ)
    final_comment = refine_comment_with_ai(input_goal, input_self_report, best_comment)

    # 3. 結果をスプレッドシートに出力
    input_sheet.update('F1', [[final_comment]])
    input_sheet.update('G1', [[f"類似度: {similarity_score:.4f}"]])
    print(f"最終結果 (F1) を出力完了。類似度: {similarity_score:.4f}")

    return final_comment, similarity_score

In [23]:
# --- 0. 必要なライブラリのインストール ---
# 警告が出ますが、無視して処理を続行してください。
!pip install -qq fugashi ipadic unidic-lite
!pip install -qq google-genai

# --- ライブラリのインポート ---
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gspread # スプレッドシートへの書き込み用
import time
import google.auth
from google.colab import auth
try:
    from google import genai
except ImportError:
    print("Warning: google-genaiのインポートに失敗しました。AI洗練機能は動作しません。")


# --- 1. 定数・初期設定 ---
# ⚠️ ここをあなたの環境に合わせて修正してください
SPREADSHEET_NAME = 'スプレッドシート作成検証'  # 正しいスプレッドシート名
TARGET_SHEET = '検索入力シート' # VBAから入力が行われるシート名
TRAINING_DATA_PATH = '/content/drive/MyDrive/AICFT/DB.csv' # 正しいDBファイルパス

# モデルのロード (以前成功した軽量モデル)
EMBEDDING_MODEL = SentenceTransformer('cl-tohoku/bert-base-japanese-whole-word-masking')

# --- 2. データの準備とエンベディング生成 ---
def load_and_embed_data(file_path):
    try:
        # 訓練データ（知識ベース）の読み込み
        df_train = pd.read_csv(file_path, encoding='utf-8', sep=',',
             usecols=['階級', '目標（50～100文字）', '自己申告（50～100文字）', '上司コメント（50～100文字）'])
        df_train.columns = ['Rank', 'Goal', 'SelfReport', 'SupervisorComment']

        # 検索対象となる結合文章を作成
        df_train['CombinedText'] = df_train['Goal'].astype(str) + " [SEP] " + df_train['SelfReport'].astype(str)

        # エンベディング生成
        embeddings = EMBEDDING_MODEL.encode(df_train['CombinedText'].tolist(), convert_to_tensor=True)
        return df_train, embeddings.cpu().numpy()
    except Exception as e:
        print(f"データ処理エラー: {e}")
        return None, None

df_train, train_embeddings = load_and_embed_data(TRAINING_DATA_PATH)

# --- 3. 検索ロジック ---
def find_best_match_comment(input_goal, input_self_report):
    if df_train is None:
        return "ERROR: Training data not loaded.", 0.0

    new_combined_text = str(input_goal) + " [SEP] " + str(input_self_report)
    new_embedding = EMBEDDING_MODEL.encode(new_combined_text, convert_to_tensor=True).cpu().numpy().reshape(1, -1)

    similarities = cosine_similarity(new_embedding, train_embeddings)[0]
    best_match_index = np.argmax(similarities)
    best_comment = df_train.iloc[best_match_index]['SupervisorComment']

    return best_comment, similarities[best_match_index]


# --- 5. AIによるコメント洗練ロジック (ラッキーテスト) ---
def refine_comment_with_ai(input_goal, input_self_report, best_comment):
    try:
        print("--- AIによるコメント洗練開始 (Gemini試行) ---")
        client = genai.Client() # APIキーなしでのアクセスを試みる

        prompt = f"""
        あなたは、人事評価の上司コメントを作成するAIです。
        以下の[入力データ]に基づき、[推薦コメント]を参考にしながら、より個性的で具体的、かつ前向きな「上司コメント」を日本語で作成してください。
        出力は洗練されたコメント本文のみとしてください。

        [入力データ]
        目標: {input_goal}
        自己申告: {input_self_report}

        [推薦コメント]
        {best_comment}
        """

        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=prompt,
        )

        refined_comment = response.text.strip()
        print("AI洗練コメント生成成功。")
        return refined_comment

    except Exception as e:
        # APIアクセス失敗時は、検索結果をそのまま返す
        print(f"AI洗練エラー。通常通り検索結果を出力します。エラー: {e}")
        return best_comment

# --- 5. AIによるコメント洗練ロジック (関数全体を削除します) ---
# def refine_comment_with_ai(...):
#     ...

# --- 4. メイン処理 (Apps Scriptから実行される想定) ---
def main_search_and_write():
    if df_train is None:
        print("検索スキップ: 訓練データなし")
        return "検索スキップ: 訓練データなし", 0.0

    print("--- gspread認証開始 ---")
    auth.authenticate_user()

    # 認証情報を取得し、gspreadを初期化
    creds, _ = google.auth.default()
    gc = gspread.authorize(creds)

    spreadsheet = gc.open(SPREADSHEET_NAME)
    input_sheet = spreadsheet.worksheet(TARGET_SHEET)

    # スプレッドシートから入力データを読み込み (B1: 目標, E1: 自己申告)
    data_list = input_sheet.get('B1:E1', value_render_option='UNFORMATTED_VALUE')

    if not data_list or len(data_list[0]) < 4:
        print("ERROR: スプレッドシートのB1～E1範囲にデータが不足しています。処理を中断します。")
        return "ERROR: データ不足", 0.0

    input_data = data_list[0]
    input_goal = input_data[0]
    input_self_report = input_data[3]

    print(f"入力データ: 目標='{input_goal}', 自己申告='{input_self_report}'")

    # 1. 意味検索実行
    final_comment, similarity_score = find_best_match_comment(input_goal, input_self_report)
    print(f"検索結果: {final_comment}")

    # 2. 結果をスプレッドシートに出力
    input_sheet.update('F1', [[final_comment]])
    input_sheet.update('G1', [[f"類似度: {similarity_score:.4f}"]])
    print(f"最終結果 (F1) を出力完了。類似度: {similarity_score:.4f}")

    return final_comment, similarity_score

# ... (続くコードは変更なし)

if __name__ == "__main__":
    # 単体テストとしてメイン処理を実行
    main_search_and_write()

--- gspread認証開始 ---
入力データ: 目標='積極的にワークライフバランスに努める', 自己申告='テレワークを５回実施した。'
検索結果: 科学的なアプローチによる隊員能力の維持向上に貢献した。プログラムの継続的な効果測定と、他の部隊への展開を検討すべき。


/tmp/ipython-input-4161856309.py:136: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  input_sheet.update('F1', [[final_comment]])
/tmp/ipython-input-4161856309.py:137: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  input_sheet.update('G1', [[f"類似度: {similarity_score:.4f}"]])


最終結果 (F1) を出力完了。類似度: 0.8999
